In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection

from sklearn import *
from sklearn.ensemble import *

from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor

from prophet import Prophet

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from matplotlib import pyplot

from tbats import BATS, TBATS
from sktime.forecasting.ets import AutoETS

from sklearn.metrics import confusion_matrix ,accuracy_score, mean_absolute_error, mean_squared_log_error

In [18]:
''' LOAD DATA
Load Data with X 'Date' Column and Y target value column. Y column must be numeric value!
Change file path specific to the location of your time series data. file path for data and loop must match exactly!
-
ARIMAdata is used so user only has to input csv file Once!
'''
                        #Change Input Data
data = pd.read_csv('data_for_inequalities.csv')
                        #Change Input Data
#loopData = pd.read_csv('data_for_inequalities.csv')

In [19]:
''' USER DEFINED
 
Length_of_split: Use 12 for month cycles or 30 for daily cycles or 24 for hourly cycles

target_col: specify target Y column name - target_col = 'Name of your target column'

number_of_steps: Specify Number of steps to predict 

length_of_train: Specify number of values to train 
 
Use # to comment out what you dont need and remove # for what you need
'''
length_of_split = 12
#   length_of_split = 30
#   length_of_split = 24
target_col = 'SG_Y'
number_of_steps = 36
length_of_train = 24

#Turns target value Y column into type integer to type float for use in XGB Loop
#loopData[target_col] = loopData[target_col].astype(float)

In [20]:
''' PRE PROCESSING 
Sorts data in sliding window technique shifting values one-step
to create new columns of X variables for target 'target_col' Y variable.
'''
for i in range(1, length_of_split+1):
    col_name = 'n-' + str(i)
    data[col_name] = data[target_col].shift(i)

data.set_index('Date', inplace = True)

In [21]:
''' DROPS NAN VALUES
'''
data = data.dropna()

In [22]:
''' TRAIN/TEST SPLIT
Splits processed data into train/test.
'''
X_train = data.head(length_of_train).drop(target_col, axis = 1)
y_train = data.head(length_of_train)[target_col]

X_test = data.tail(length_of_split).drop(target_col, axis = 1)
y_test = data.tail(length_of_split)[target_col]

In [23]:
X_train

,n-1,n-2,n-3,n-4,n-5,n-6,n-7,n-8,n-9,n-10,n-11,n-12
Date,,,,,,,,,,,,
2018/12,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286,89.916667,88.700000,86.125000,85.500000,82.750000,76.500000
2019/01,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286,89.916667,88.700000,86.125000,85.500000,82.750000
2019/02,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286,89.916667,88.700000,86.125000,85.500000
2019/03,87.791667,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286,89.916667,88.700000,86.125000
2019/04,87.541667,87.791667,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286,89.916667,88.700000
2019/05,85.791667,87.541667,87.791667,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286,89.916667
2019/06,86.375000,85.791667,87.541667,87.791667,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500,89.214286
2019/07,88.125000,86.375000,85.791667,87.541667,87.791667,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333,89.062500
2019/08,88.125000,88.125000,86.375000,85.791667,87.541667,87.791667,85.708333,84.375000,84.375000,83.772727,83.550000,84.833333


In [24]:
y_train

Date
2018/12     84.375000
2019/01     85.708333
2019/02     87.791667
2019/03     87.541667
2019/04     85.791667
2019/05     86.375000
2019/06     88.125000
2019/07     88.125000
2019/08     89.708333
2019/09     91.625000
2019/10     92.291667
2019/11     95.708333
2019/12     96.750000
2020/01     96.166667
2020/02     94.833333
2021/03     93.833333
2021/04     96.083333
2021/05     98.000000
2021/06     98.500000
2021/07    104.000000
2021/08    104.333333
2021/09    103.833333
2021/10    104.416667
2021/11    103.916667
Name: SG_Y, dtype: float64

In [25]:
#Create estimator and mae score array to be appended with models in ensemble 
estimators=[]
mae_scores=[]

In [26]:

#Build first base model - XGBOOST
model1 = XGBRegressor()
#Append estimators array by adding model1
estimators.append(("XGB",model1))
#Fit model on training data 
model1.fit(X_train, y_train)
#Forecast
target_pred1 = model1.predict(X_test)
#Calculate MAE Score for model1
XGBmae = mean_absolute_error(y_test, target_pred1)
#Calculate RMSLE score for ensemble model
XGBRMSLE = mean_squared_log_error(y_test, target_pred1, squared=False)
#Print scores for model1
print("XGB MAE: %.3f" % XGBmae )
print("XGB RMSLE: %.3f" % XGBRMSLE)
#Append mae_scores array by adding model1 mae score 
mae_scores.append(XGBmae)

XGB MAE: 2.688
XGB RMSLE: 0.030


In [27]:
#Build second base model for ensemble - CatBoost
model2 = CatBoostRegressor()
#Append estimators array by adding model2
estimators.append(("CatBoost",model2))
#Fit model on training data
model2.fit(X_train, y_train)
#Forecast
target_pred2 = model2.predict(X_test)
#Calculate MAE score for model2
CBmae = mean_absolute_error(y_test, target_pred2)
#Calculate RMSLE score for model2
CatBRMSLE = mean_squared_log_error(y_test, target_pred2, squared=False)
#Print scores for model2
print("CatBoost MAE: %.3f" % CBmae)
print("CatBoost RMSLE: %.3f" % CatBRMSLE)
#Append mae_scores array by adding model2 mae score 
mae_scores.append(CBmae)

Learning rate set to 0.022712
0:	learn: 6.4857650	total: 814us	remaining: 814ms
1:	learn: 6.4114824	total: 1.43ms	remaining: 712ms
2:	learn: 6.3515392	total: 2.1ms	remaining: 698ms
3:	learn: 6.2868916	total: 2.55ms	remaining: 635ms
4:	learn: 6.2121332	total: 2.98ms	remaining: 593ms
5:	learn: 6.1497907	total: 3.17ms	remaining: 526ms
6:	learn: 6.0820072	total: 3.59ms	remaining: 510ms
7:	learn: 6.0197156	total: 3.92ms	remaining: 486ms
8:	learn: 5.9534062	total: 4.35ms	remaining: 479ms
9:	learn: 5.8785473	total: 4.76ms	remaining: 471ms
10:	learn: 5.8170490	total: 5.09ms	remaining: 458ms
11:	learn: 5.7421993	total: 5.58ms	remaining: 460ms
12:	learn: 5.6704725	total: 6.05ms	remaining: 459ms
13:	learn: 5.6081586	total: 6.49ms	remaining: 457ms
14:	learn: 5.5415900	total: 6.69ms	remaining: 440ms
15:	learn: 5.4809782	total: 8.4ms	remaining: 517ms
16:	learn: 5.4279420	total: 9.38ms	remaining: 542ms
17:	learn: 5.3775967	total: 10.1ms	remaining: 553ms
18:	learn: 5.3264636	total: 10.7ms	remaining: 5

In [28]:
#Build third base model for ensemble - Decision Tree
model3 = DecisionTreeRegressor()
#Append estimators array by adding model3
estimators.append(("Decision Tree",model3))
#Fit model on training data
model3.fit(X_train, y_train)
#Forecast
target_pred3 = model3.predict(X_test)
#Calculate MAE score for model3
DTreemae = mean_absolute_error(y_test, target_pred3)
#Calculate RMSLE score for model3
DTreeRMSLE = mean_squared_log_error(y_test, target_pred3, squared=False)
#Print scores for model3
print("Decision Tree MAE: %.3f" % DTreemae)
print("Decision Tree RMSLE: %.3f" % DTreeRMSLE)
#Append mae_scores array by adding model3 mae score 
mae_scores.append(DTreemae)

Decision Tree MAE: 2.587
Decision Tree RMSLE: 0.029


In [29]:
#Stacking
#Define meta learner model
level1 = LinearRegression()
#Define the stacking ensemble
ensemble_model = StackingRegressor(estimators=estimators, final_estimator=level1, cv=5)
#Fit model on training data 
er = ensemble_model.fit(X_train, y_train)
#Forecast
target_pred = er.predict(X_test)
#Calculate MAE score for ensemble model
Ensemblemae = mean_absolute_error(y_test, target_pred)
#Calculate RMSLE score for ensemble model
EnsembleRMSLE = mean_squared_log_error(y_test, target_pred, squared=False)
#Print scores for model3
print("Ensemble MAE: %.3f" % Ensemblemae)
print("Ensemble RMSLE: %.3f" % EnsembleRMSLE)
print(target_pred)

Learning rate set to 0.022712
0:	learn: 6.4857650	total: 658us	remaining: 658ms
1:	learn: 6.4114824	total: 1.13ms	remaining: 565ms
2:	learn: 6.3515392	total: 1.56ms	remaining: 520ms
3:	learn: 6.2868916	total: 1.97ms	remaining: 490ms
4:	learn: 6.2121332	total: 2.37ms	remaining: 472ms
5:	learn: 6.1497907	total: 2.55ms	remaining: 423ms
6:	learn: 6.0820072	total: 2.98ms	remaining: 423ms
7:	learn: 6.0197156	total: 3.3ms	remaining: 409ms
8:	learn: 5.9534062	total: 3.73ms	remaining: 411ms
9:	learn: 5.8785473	total: 4.13ms	remaining: 409ms
10:	learn: 5.8170490	total: 4.45ms	remaining: 400ms
11:	learn: 5.7421993	total: 5.15ms	remaining: 424ms
12:	learn: 5.6704725	total: 5.65ms	remaining: 429ms
13:	learn: 5.6081586	total: 6.12ms	remaining: 431ms
14:	learn: 5.5415900	total: 6.33ms	remaining: 415ms
15:	learn: 5.4809782	total: 6.88ms	remaining: 423ms
16:	learn: 5.4279420	total: 7.38ms	remaining: 427ms
17:	learn: 5.3775967	total: 7.83ms	remaining: 427ms
18:	learn: 5.3264636	total: 8.3ms	remaining: 4

In [30]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':target_pred})
df

,Actual,Predicted
Date,,
2021/12,104.250000,106.082803
2022/01,103.458333,105.912608
2022/02,103.916667,106.343353
2022/03,108.541667,106.265241
2022/04,108.458333,105.838701
2022/05,107.291667,105.905717
2022/06,106.583333,105.894422
2022/07,101.208333,105.894422
2022/08,100.125000,106.308463


In [31]:
#print accuracy score for ensemble 
print("MAE of ensemble model is : %.3f" % Ensemblemae)
print("RMSLE of ensemble model is: %.3f" % EnsembleRMSLE)

MAE of ensemble model is : 3.314
RMSLE of ensemble model is: 0.036


In [32]:
#Evaluation side by side
print("XGB RMSLE          : %.4f" % XGBRMSLE)
print("CatBoost RMSLE         : %.4f" % CatBRMSLE)
print("Decision Tree RMSLE: %.4f" % DTreeRMSLE)
print("Ensemble RMSLE     : %.4f" % EnsembleRMSLE)

XGB RMSLE          : 0.0298
CatBoost RMSLE         : 0.0291
Decision Tree RMSLE: 0.0291
Ensemble RMSLE     : 0.0361
